In [2]:
import praw
from datetime import datetime, timedelta, timezone
import math

In [3]:
client_id = "e8O8uDpRkFbES6y2ZA9XwQ"
client_secret="5LSSWHnkHVM5-BIkxpgYlu8fKIqnzw"
user_agent = "Decaying Window"

In [4]:

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [5]:
def calculate_score(interactions, time_weight):
    """
    Calculate the score of a post based on its interactions and time weight.
    """
    return interactions * time_weight

In [6]:
def find_popular_posts(c, threshold, subreddit_name, time_window_hours):
    """
    Find popular posts in a subreddit within a specified time window.
    """
    subreddit = reddit.subreddit(subreddit_name)

    if time_window_hours <= 1:
        time_filter = 'hour'
    elif time_window_hours <= 24:
        time_filter = 'day'
    elif time_window_hours <= 168:
        time_filter = 'week'
    elif time_window_hours <= 720:
        time_filter = 'month'
    else:
        time_filter = 'year'

    current_time = datetime.now(timezone.utc)
    start_time = current_time - timedelta(hours=time_window_hours)

    popular_posts = []

    for post in subreddit.top(time_filter=time_filter, limit=None):
        post_time = datetime.fromtimestamp(post.created_utc, tz=timezone.utc)
        if post_time >= start_time:
            time_weight = math.ceil((1 - c) ** ((current_time - post_time).total_seconds() / 3600 + 1))
            score = calculate_score(post.score, time_weight)
            if score >= threshold:
                popular_posts.append((post.title, score))

    popular_posts.sort(key=lambda x: x[1], reverse=True)
    return popular_posts

In [10]:
c = 1e-6
threshold = 10
subreddit_name = 'philosophy'
time_window_hours = 51

In [11]:
popular_posts = find_popular_posts(c, threshold, subreddit_name, time_window_hours)

In [12]:
print(f"Popular posts in r/{subreddit_name} within the last {time_window_hours} hour(s):")
for i, (title, score) in enumerate(popular_posts, start=1):
    print(f"{i}. {title} - Score: {score}")

Popular posts in r/philosophy within the last 51 hour(s):
1. Over-enthusiasm for AI is born out of a misguided hope of "democratizing" intelligence - Score: 520
2. Quick introduction to how 6 major philosophies (Aristotle, Epicurus, the Stoics, the Buddha, Confucianism, and Existentialism) define and approach the good life, with links to further reading - Score: 78
3. How it would feel to be a rat under remote control: analyzing a breakdown of the connection between mind and body - Score: 40
4. Philosophy of mathematics: Structuralism - Joel David Hamkins - Score: 33
5. In sections 1 and 2 of Being and Time, Martin Heidegger explores why it is important to try to uncover the meaning of Being and the way in which its investigation should be structured, introducing his infamous concept of Dasein in the process - Score: 13
